# Imports

In [1]:
import pandas as pd
import os
import PyPDF2
import re

# Playoff Outcomes Parsing

In [8]:
# Define the file path
csv_path = r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Data\Playoff_Outcomes.csv"
output_path = r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Data\Parsed_Playoff_Outcomes.csv"

# Parse playoff series CSV
def parse_playoff_csv(csv_path):
    # Ensure directory exists
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    # Check if file exists
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"The file '{csv_path}' was not found. Please verify the path or create the file.")
    if os.path.getsize(csv_path) == 0:
        raise ValueError(f"The file '{csv_path}' is empty")
    
    # Read CSV
    try:
        df = pd.read_csv(csv_path, header=0)
    except pd.errors.EmptyDataError:
        raise ValueError(f"The file '{csv_path}' contains no data to parse")
    
    if df.empty:
        raise ValueError(f"No data parsed from '{csv_path}' - check file content")
    
    print(f"Initial DataFrame shape: {df.shape}")
    print("Initial columns:", df.columns.tolist())
    print("First few rows of raw data:")
    print(df.head())
    
    # Rename columns and drop empty ones
    df.columns = [
        "Year", "League", "Series_Name", "Series_Dates", "Empty1", 
        "Winner_Team", "Winner_Wins", "Empty2", 
        "Loser_Team", "Loser_Wins", "Empty3", 
        "Favorite", "Underdog"
    ]
    df = df.drop(columns=["Empty1", "Empty2", "Empty3"])
    print(f"After dropping empty columns, shape: {df.shape}")
    
    # Extract Winner Seed and clean Winner_Team
    df["Winner_Seed"] = df["Winner_Team"].str.extract(r"\((\d+)\)", expand=False).fillna(0).astype(int)
    df["Winner_Team"] = df["Winner_Team"].str.replace(r"\s\(\d+\)", "", regex=True)
    print(f"After Winner_Seed extraction, shape: {df.shape}")
    
    # Extract Loser Seed and clean Loser_Team
    df["Loser_Seed"] = df["Loser_Team"].str.extract(r"\((\d+)\)", expand=False).fillna(0).astype(int)
    df["Loser_Team"] = df["Loser_Team"].str.replace(r"\s\(\d+\)", "", regex=True)
    print(f"After Loser_Seed extraction, shape: {df.shape}")
    
    # Extract odds
    df["Favorite_Team"] = df["Favorite"].str.extract(r"([A-Z]{2,3})", expand=False)
    df["Favorite_Odds"] = df["Favorite"].str.extract(r"([-+]?\d+)", expand=False).astype(float)
    df["Underdog_Team"] = df["Underdog"].str.extract(r"([A-Z]{2,3})", expand=False)
    df["Underdog_Odds"] = df["Underdog"].str.extract(r"([+-]?\d+)", expand=False).astype(float)
    df = df.drop(columns=["Favorite", "Underdog"])
    print(f"Final DataFrame shape: {df.shape}")
    
    return df

# Run the parsing with error handling
try:
    playoff_df = parse_playoff_csv(csv_path)
    print("\nPlayoff DataFrame:")
    print(playoff_df)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    playoff_df.to_csv(output_path, index=False)
    print(f"Saved to {output_path} with shape {playoff_df.shape}")
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")
except ValueError as e:
    print(f"ValueError: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Initial DataFrame shape: (949, 13)
Initial columns: ['Yr', 'Lg', 'Series', 'Unnamed: 3', 'Unnamed: 4', 'Winning Team', "Winner W's", 'Unnamed: 7', 'Losing Team', "Loser W's", 'Unnamed: 10', 'Favorite', 'Underdog']
First few rows of raw data:
     Yr   Lg                    Series            Unnamed: 3  Unnamed: 4  \
0  2024  NBA  Eastern Conf First Round   Apr 21 - May 1 2024         NaN   
1  2024  NBA  Eastern Conf First Round   Apr 20 - May 5 2024         NaN   
2  2024  NBA  Eastern Conf First Round   Apr 21 - May 2 2024         NaN   
3  2024  NBA  Eastern Conf First Round   Apr 20 - May 2 2024         NaN   
4  2024  NBA  Western Conf First Round  Apr 21 - Apr 29 2024         NaN   

                Winning Team  Winner W's  Unnamed: 7  \
0         Boston Celtics (1)           4         NaN   
1    Cleveland Cavaliers (4)           4         NaN   
2         Indiana Pacers (6)           4         NaN   
3        New York Knicks (2)           4         NaN   
4  Oklahoma City Thun

# Advanced Stats Parsing

In [3]:
# Define file paths
pdf_path = r"C:/Users/jonla/NBA_Playoffs_Series_Predictor/Data/Advanced_Stats_96-24.pdf"
csv_path = r"C:/Users/jonla/NBA_Playoffs_Series_Predictor/Data/Advanced_Stats_96-24.csv"

# Define column headers
columns = [
    "Season", "TEAM", "GP", "W", "L", "MIN", "OffRtg", "DefRtg", "NetRtg", "AST%", 
    "AST/TO", "AST Ratio", "OREB%", "DREB%", "REB%", "TOV%", "eFG%", "TS%", "PACE", "PIE", "POSS"
]

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + " "
        return text
    except FileNotFoundError:
        raise FileNotFoundError(f"PDF file not found at {pdf_path}")
    except Exception as e:
        raise Exception(f"Error reading PDF: {e}")

# Process text to extract season-separated data
def process_text(text):
    tokens = [token for token in text.split() if token.strip()]
    season_data = {}
    current_season = None
    current_tokens = []
    
    season_pattern = re.compile(r"^\d{4}-\d{2}$")
    
    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        if season_pattern.match(token):
            if current_season and current_tokens:
                season_data[current_season] = current_tokens
            current_season = token
            current_tokens = []
            i += 1
            continue
        
        if current_season:
            current_tokens.append(token)
        
        i += 1
    
    if current_season and current_tokens:
        season_data[current_season] = current_tokens
    
    # Parse each season’s tokens into a DataFrame
    dataframes = {}
    all_data = []
    
    nba_team_keywords = [
        "Bulls", "Jazz", "SuperSonics", "Hornets", "Pistons", "Clippers", "Lakers", "Rockets",
        "Knicks", "Celtics", "Heat", "Spurs", "Pacers", "Warriors", "Nuggets", "Raptors",
        "Wizards", "Cavaliers", "Suns", "Trail Blazers", "Kings", "Mavericks", "Thunder",
        "Timberwolves", "Grizzlies", "Nets", "Pelicans", "Magic", "76ers", "Bucks", "Hawks"
    ]
    
    for season, season_tokens in season_data.items():
        structured_data = []
        j = 0
        skipped_rows = []
        
        while j < len(season_tokens):
            token = season_tokens[j]
            
            skip_tokens = [
                "Glossary", "Share", "Rows", "Page", "of", "TEAM", "•", "AST", "Ratio", "OREB%", 
                "DREB%", "REB%", "TOV%", "eFG%", "TS%", "PACE", "PIE", "POSS", "GP", "W", "L", 
                "MIN", "OffRtg", "DefRtg", "NetRtg", "AST%", "AST/TO"
            ]
            if token in skip_tokens or (token.isupper() and len(token) > 3 and token not in ["W", "L"]):
                j += 1
                continue
            
            if token.isdigit() and 1 <= int(token) <= 30:
                j += 1
                team_parts = []
                start_j = j
                while j < len(season_tokens):
                    next_token = season_tokens[j]
                    if next_token.isdigit() and 48 <= int(next_token.replace(",", "").replace(".", "")) <= 82:
                        break
                    if next_token in skip_tokens or (next_token.isdigit() and int(next_token) < 48):
                        j += 1
                        continue
                    team_parts.append(next_token)
                    j += 1
                
                team_name = " ".join(team_parts).strip()
                if not team_name or (not any(keyword in team_name for keyword in nba_team_keywords) and len(team_parts) < 2):
                    skipped_rows.append(f"Skipped at {start_j}: {team_name} (tokens: {season_tokens[start_j:j+5]})")
                    j += 1
                    continue
                
                stats = []
                for _ in range(19):
                    if j < len(season_tokens):
                        stats.append(season_tokens[j])
                        j += 1
                    else:
                        stats.append(None)
                
                row = [season, team_name] + stats
                structured_data.append(row)
                all_data.append(row)
                continue
            
            j += 1
        
        df = pd.DataFrame(structured_data, columns=columns)
        for col in columns[2:]:
            df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors="coerce")
        dataframes[season] = df
        
        if season == "2000-01":
            print(f"\n2000-01 Season: {len(df)} teams extracted")
            print("Skipped rows:", skipped_rows)
    
    series_df = pd.DataFrame(all_data, columns=columns)
    for col in columns[2:]:
        series_df[col] = pd.to_numeric(series_df[col].str.replace(",", ""), errors="coerce")
    
    print(f"\nExtracted {len(dataframes)} seasons")
    print(f"Total rows across all seasons: {len(series_df)}")
    return dataframes, series_df

# Save to CSV
def save_to_csv(series_df, csv_path):
    try:
        series_df.to_csv(csv_path, index=False)
        print(f"Saved all seasons to {csv_path} with shape {series_df.shape}")
    except Exception as e:
        print(f"Error saving to {csv_path}: {e}")

# Run the extraction and processing
try:
    text = extract_text_from_pdf(pdf_path)
    dataframes, series_df = process_text(text)
    
    # Rename "Season" to "Year" for consistency with playoff_df
    series_df.rename(columns={'Season': 'Year'}, inplace=True)
    
    # Print info
    print("\nFirst few rows of 2000-01 season:")
    print(dataframes["2000-01"].head(10))
    print("\nFirst few rows of master DataFrame:")
    print(series_df.head())
    
    # Save to CSV
    save_to_csv(series_df, csv_path)
except Exception as e:
    print(f"Error in Advanced Stats Parsing: {e}")


2000-01 Season: 29 teams extracted
Skipped rows: []

Extracted 28 seasons
Total rows across all seasons: 832

First few rows of 2000-01 season:
    Season                    TEAM  GP   W   L     MIN  OffRtg  DefRtg  \
0  2000-01      Los Angeles Lakers  82  56  26  3981.0   107.0   103.6   
1  2000-01         Milwaukee Bucks  82  52  30  3956.0   106.8   102.6   
2  2000-01               Utah Jazz  82  53  29  3956.0   106.1   100.9   
3  2000-01        Dallas Mavericks  82  53  29  3961.0   105.6   101.2   
4  2000-01         Houston Rockets  82  45  37  3966.0   105.2   102.8   
5  2000-01       San Antonio Spurs  82  58  24  3966.0   105.0    96.6   
6  2000-01  Portland Trail Blazers  82  50  32  3956.0   104.7   100.5   
7  2000-01     Seattle SuperSonics  82  44  38  3961.0   104.4   104.2   
8  2000-01         Toronto Raptors  82  47  35  3991.0   104.4   102.0   
9  2000-01        Sacramento Kings  82  55  27  4016.0   104.2    98.5   

   NetRtg  AST%  ...  AST Ratio  OREB%  

# Merge Playoff Series and Advanced stats to create the Master_df

In [4]:
output_path = r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Data\Master_DF.csv"

# Verify column names
print("playoff_df columns:", playoff_df.columns.tolist())
print("series_df columns:", series_df.columns.tolist())

# Check Year format for debugging
print("Sample playoff_df['Year'] values:", playoff_df["Year"].head().tolist())
print("Sample series_df['Year'] values:", series_df["Year"].head().tolist())

# Standardize the 'Year' column in series_df
# Convert '1996-97' to 1997, '2023-24' to 2024, etc.
def standardize_year(year):
    if isinstance(year, str) and '-' in year:
        end_year = int(year.split('-')[1])
        # For years like '1996-97', end_year is 97 → 1997
        # For years like '2023-24', end_year is 24 → 2024
        return end_year + 2000 if end_year < 50 else end_year + 1900
    return year

series_df["Year"] = series_df["Year"].apply(standardize_year).astype(int)

# Verify standardized Year values
print("Sample series_df['Year'] values after standardization:", series_df["Year"].head().tolist())

# Standardize team names
team_name_mapping = {
    "Trail Blazers": "Portland Trail Blazers",
    "SuperSonics": "Seattle SuperSonics",
    "76ers": "Philadelphia 76ers",
    "Nets": "New Jersey Nets",
    "Hornets": "Charlotte Hornets",
    "Pelicans": "New Orleans Pelicans",
    "Thunder": "Oklahoma City Thunder",
    "Knicks": "New York Knicks",
    "Celtics": "Boston Celtics",
    "Heat": "Miami Heat",
    "Bucks": "Milwaukee Bucks",
    "Cavaliers": "Cleveland Cavaliers",
    "Pacers": "Indiana Pacers",
    "Magic": "Orlando Magic",
    "Bobcats": "Charlotte Bobcats",
    "Bullets": "Washington Bullets",
    "Spurs": "San Antonio Spurs",
    "Grizzlies": "Vancouver Grizzlies",
    "Sonics": "Seattle SuperSonics",
}
playoff_df["Winner_Team"] = playoff_df["Winner_Team"].replace(team_name_mapping)
playoff_df["Loser_Team"] = playoff_df["Loser_Team"].replace(team_name_mapping)
series_df["TEAM"] = series_df["TEAM"].replace(team_name_mapping)

# Merge for Winner_Team stats
winner_stats = series_df.copy()
winner_stats = winner_stats.rename(columns=lambda x: f"Winner_{x}" if x != "Year" and x != "TEAM" else x)
master_df = pd.merge(
    playoff_df,
    winner_stats,
    how="left",
    left_on=["Year", "Winner_Team"],
    right_on=["Year", "TEAM"],
    suffixes=("", "_winner")
).drop(columns=["TEAM"], errors="ignore")

# Merge for Loser_Team stats
loser_stats = series_df.copy()
loser_stats = loser_stats.rename(columns=lambda x: f"Loser_{x}" if x != "Year" and x != "TEAM" else x)
master_df = pd.merge(
    master_df,
    loser_stats,
    how="left",
    left_on=["Year", "Loser_Team"],
    right_on=["Year", "TEAM"],
    suffixes=("", "_loser")
).drop(columns=["TEAM"], errors="ignore")

# Finalize master_df
base_columns = [
    "Year", "League", "Series_Name", "Series_Dates",
    "Winner_Team", "Winner_Wins", "Loser_Team", "Loser_Wins",
    "Winner_Seed", "Loser_Seed", "Favorite_Team", "Favorite_Odds",
    "Underdog_Team", "Underdog_Odds"
]
winner_stat_columns = [col for col in master_df.columns if col.startswith("Winner_") and col not in base_columns]
loser_stat_columns = [col for col in master_df.columns if col.startswith("Loser_") and col not in base_columns]

master_df = master_df[base_columns + winner_stat_columns + loser_stat_columns]

# Verify and save
print("Master DataFrame shape:", master_df.shape)
print("\nFirst few rows of master_df:")
print(master_df.head())
print("\nColumns in master_df:", master_df.columns.tolist())

print("Rows with missing Winner stats:", master_df[winner_stat_columns].isna().all(axis=1).sum())
print("Rows with missing Loser stats:", master_df[loser_stat_columns].isna().all(axis=1).sum())

unmatched_winners = master_df[master_df["Winner_GP"].isna()][["Year", "Winner_Team"]]
unmatched_losers = master_df[master_df["Loser_GP"].isna()][["Year", "Loser_Team"]]
print("\nUnmatched Winner_Team:\n", unmatched_winners)
print("\nUnmatched Loser_Team:\n", unmatched_losers)

try:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    master_df.to_csv(output_path, index=False)
    print(f"Saved to {output_path}")
except PermissionError as e:
    print(f"PermissionError: Unable to write to {output_path}. {e}")
except Exception as e:
    print(f"Unexpected error saving to {output_path}: {e}")

playoff_df columns: ['Year', 'League', 'Series_Name', 'Series_Dates', 'Winner_Team', 'Winner_Wins', 'Loser_Team', 'Loser_Wins', 'Winner_Seed', 'Loser_Seed', 'Favorite_Team', 'Favorite_Odds', 'Underdog_Team', 'Underdog_Odds']
series_df columns: ['Year', 'TEAM', 'GP', 'W', 'L', 'MIN', 'OffRtg', 'DefRtg', 'NetRtg', 'AST%', 'AST/TO', 'AST Ratio', 'OREB%', 'DREB%', 'REB%', 'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'POSS']
Sample playoff_df['Year'] values: [2024, 2024, 2024, 2024, 2024]
Sample series_df['Year'] values: ['1996-97', '1996-97', '1996-97', '1996-97', '1996-97']
Sample series_df['Year'] values after standardization: [1997, 1997, 1997, 1997, 1997]
Master DataFrame shape: (949, 52)

First few rows of master_df:
   Year League               Series_Name          Series_Dates  \
0  2024    NBA  Eastern Conf First Round   Apr 21 - May 1 2024   
1  2024    NBA  Eastern Conf First Round   Apr 20 - May 5 2024   
2  2024    NBA  Eastern Conf First Round   Apr 21 - May 2 2024   
3  2024    NBA 